# Economic Dispatch

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\fenya\Documents\BEE4750\Project\FP-Group6`


In [ ]:
Pkg.add("Plots")

: 

In [ ]:
Pkg.add("CSV")
Pkg.add("MarkdownTables")
Pkg.add("Dates")

In [ ]:
using JuMP
using HiGHS
using DataFrames
using Plots
using Measures
using CSV
using MarkdownTables
# using NamedArrays
using Dates

In [ ]:
# load the data, pull Zone C, and reformat the DataFrame
CU_edemand = DataFrame(CSV.File("data/Cornell_Electricity_Data.csv"))
rename!(CU_edemand, :"slottime_GMT" => :Date)
edemand = CU_edemand[:, [:Date, :slotavg]]
rename!(edemand, :slotavg => :edemand)
# demand[:, :Hour] = 1:nrow(demand)

# plot demand
plot(edemand.Date, edemand.edemand, xlabel="Day of Year", ylabel="Demand (MWh)", label=:false)

In [ ]:
# load steam data for heat demand
CU_hdemand = DataFrame(CSV.File("data/Cornell_steam_data.csv"))
rename!(CU_hdemand, :"slottime_GMT" => :Date)
hdemand = CU_hdemand[:, [:Date, :slotavg]]
rename!(hdemand, :slotavg => :hdemand)

# Converting to units of MMBTU of heat produced (before distribution losses)
hdemand.hdemand = hdemand.hdemand*0.001194
# demand[:, :Hour] = 1:nrow(demand)

# plot demand
plot(hdemand.Date, hdemand.hdemand, title="load steam data for heat demand", xlabel="Day of Year", ylabel="Total heat as steam delivered (MMBTU/hour)", label=:false)

In [ ]:
# Information about different generation and heating sources
# gens is electricity - natural gas is CHP 
# -- heat pumps run on electricity and produce heat
gens = DataFrame(CSV.File("data/Gen_data_CU.csv"))

# heat is heat
heat = DataFrame(CSV.File("data/Heat_data_CU.csv"))

In [ ]:
gens

In [ ]:
# Electricity production conversion (MWh/MMBtu), assumed constant
# Natural gas - electricity produced per heat
# Heat Pumps - electricity required per heat
heat.Conversion_Factor

Finally, we load the hourly solar and wind capacity factors, which are
plotted in <a href="#fig-cf" class="quarto-xref">Figure 2</a>. These
tell us the fraction of installed capacity which is expected to be
available in a given hour for generation (typically based on the average
meteorology).

In [ ]:
[Dates.DateTime(edemand.Date[i], dateformat"yyyy-mm-dd HH:MM:SS") for i in 1:length(edemand.Date)]

In [ ]:
transform(edemand, :Date => (ByRow(t -> Dates.DateTime(t, dateformat"yyyy-mm-dd HH:MM:SS"))) => :Date)

In [ ]:
cap_factor = DataFrame(CSV.File("data/2022_solar_CF_zone_C.csv", header=4))
cap_factor=  cap_factor[:, [:time, :electricity]]
rename!(cap_factor, :electricity => :Solar)

# plot January capacity factors
# p1 = plot(cap_factor.Wind[1:(24*31)], label="Wind")
p1 = plot(cap_factor.Solar[1:(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")

# p2 = plot(cap_factor.Wind[4344:4344+(24*31)], label="Wind")
p2 = plot(cap_factor.Solar[4344:4344+(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")

display(p1)
display(p2)

## Drop missing data

In [ ]:
e_missing_dates = setdiff(DateTime(2022, 1, 1, 0):Hour(1):DateTime(2022, 12, 31, 23),[Dates.DateTime(edemand.Date[i], dateformat"yyyy-mm-dd HH:MM:SS") for i in 1:length(edemand.Date)])
h_missing_dates = setdiff(DateTime(2022, 1, 1, 0):Hour(1):DateTime(2022, 12, 31, 23),[Dates.DateTime(hdemand.Date[i], dateformat"yyyy-mm-dd HH:MM:SS") for i in 1:length(hdemand.Date)])
cap_factor = DataFrame(CSV.File("data/2022_solar_CF_zone_C.csv", header=4))
cap_factor=  cap_factor[:, [:time, :electricity]]
rename!(cap_factor, :electricity => :Solar)
missing_dates = [h_missing_dates; e_missing_dates]
e_dts = [Dates.DateTime(edemand.Date[i], dateformat"yyyy-mm-dd HH:MM:SS") for i in 1:length(edemand.Date)]
h_dts = [Dates.DateTime(hdemand.Date[i], dateformat"yyyy-mm-dd HH:MM:SS") for i in 1:length(hdemand.Date)]
cf_dts = [Dates.DateTime(cap_factor.time[i], dateformat"mm/dd/yyyy HH:MM") for i in 1:length(cap_factor.time)]
cap_factor = cap_factor[[cf_dts[i] ∉ missing_dates for i in 1:length(cap_factor.time)],:]
edemand = edemand[[e_dts[i] ∉ missing_dates for i in 1:length(edemand.Date)],:]
hdemand = hdemand[[h_dts[i] ∉ missing_dates for i in 1:length(hdemand.Date)],:]

## Solutions

Base case capacity expansion (no heating)

Decision variables:
$x_{g}$ = Installed capacity for generator type g (MW)

$y_{g,t}$ = Production from generator g in time t (MWh)

$NSE_{t}$ = non-served energy in time t (MWh)

Minimize total cost = fixed cost + variable cost + unserved energy cost
\begin{align}
\min_{x, y, NSE} \quad & \sum_{g \in \mathcal{G}} \text{FixedCost}_g \times x_g + \sum_{t \in \mathcal{T}} \sum_{g \in \mathcal{G}} \text{VarCost}_g \times y_{g,t} & \\
& \quad + \sum_{t \in \mathcal{T}} \text{NSECost} \times NSE_t & \\[0.5em]
\text {subject to:} \quad & \sum_{g \in \mathcal{G}} y_{g,t} + NSE_t \geq d_t \qquad \forall t \in \mathcal{T} \\[0.5em]
\text{(Meeting demand in each hour)} \\
& y_{g,t} \leq x_g*c_{g,t} \qquad \qquad \qquad\qquad  \forall g \in {G},  \forall t \in \mathcal{T} \\[0.5em]
\text{(Generator capacity limits)} \\
& x_g, y_{g,t}, NSE_t \geq 0 \qquad \qquad \forall g \in {G},  \forall t \in \mathcal{T}
\end{align}

Economic Dispatch with CHP

Decision variables:

$y_{g,t}$ = Production from generator g in time t (MWh)

$z_{s,t}$ = Heat production from source s in time t (MWh)

$NSE_{t}$ = non-served energy in time t (MWh)

Parameters:

r = Conversion factor for CHP, MWh produced per MMBTU produced

$c_{hp,t}$ = Conversion factor for heat pump, MWh required per MMBTU produced


Minimize total cost = fixed cost + variable cost + unserved energy cost
\begin{align}
\sum_{t \in \mathcal{T}} \sum_{g \in \mathcal{G}} \text{VarCost}_g \times y_{g,t} \quad + \sum_{t \in \mathcal{T}} \text{NSECost} \times NSE_t & \\[0.5em]
\text {subject to:} \quad & \sum_{g \in \mathcal{G}} y_{g,t} + NSE_t \geq d_t + c_{hp,t}* z_{hp,t} \qquad \forall t \in \mathcal{T} \\[0.5em]
\text{(Meeting demand in each hour)} \\
\quad & \sum_{s \in \mathcal{S}} h_{s,t} \geq zd_t \qquad \forall t \in \mathcal{T} \\[0.5em]
\text{(Meeting heating demand in each hour)} \\
\quad & z_{n,t} = r*y_{n,t} \qquad \forall t \in \mathcal{T} \\[0.5em]
\text{(Assume constant ratio of heat to electricty production from natural gas)} \\
& y_{g,t} \leq x_g*c_{g,t} \qquad \qquad \qquad\qquad  \forall g \in {G},  \forall t \in \mathcal{T} \\[0.5em]
\text{(Generator capacity limits)} \\
& x_g, y_{g,t}, NSE_t \geq 0 \qquad \qquad \forall g \in {G},  \forall t \in \mathcal{T}
\end{align}

In [ ]:
# capacity factor for gas will be 1 --> this sets the capacity factor as 1 for every day of the year
cap_factor[:, :Gas] .=1

# cap_factor[:, :Geothermal] .=0.8
# cap_factor[:, :NG_CCGT] .=1
# cap_factor[:, :NG_CT] .=1

# all capacity factors together into one data frame
select!(cap_factor, :Gas, :Solar)

In [ ]:
# No heat, Just electricity no heatpumps
# define sets
G = 1:nrow(gens)
T = 1:nrow(edemand)
NSECost = 10000

gencap = Model(HiGHS.Optimizer)
# define variables
@variables(gencap, begin
    y[g in G, t in T] >= 0
    NSE[t in T] >= 0
end)

In [ ]:
# Optimizing cost (cost is constant over time per mwh) of just electricity

# define sets
G = 1:nrow(gens)
T = 1:nrow(edemand)
NSECost = 10000

gencap = Model(HiGHS.Optimizer)
# define variables
@variables(gencap, begin
    y[g in G, t in T] >= 0
    NSE[t in T] >= 0
end)

@objective(gencap, Min, 
   sum(gens[G, :VarCost] .* sum(y[:, t] for t in T)) + NSECost * sum(NSE)
)

@constraint(gencap, load[t in T], sum(y[:, t]) + NSE[t] >= edemand.edemand[t])

@constraint(gencap, availability[g in G, t in T], y[g, t] <= gens[g, :Capacity]*cap_factor[t,g])
optimize!(gencap)



In [ ]:
@show objective_value(gencap)

Objective value wihtout heat is $4476878

In [ ]:
# Plotting natural gas and solar generation at the optimized cost
# This is just optimizing to meet electricity demand

gen = value.(y).data 
p = areaplot(gen'[1:200,:], 
    label=permutedims(gens[:, :Column1]), 
    xlabel = "Hour", 
    ylabel ="Generated Electricity (MW)", 
    color_palette=:mk_15,
    grid=:false,
    # ylim=(0, 100),
)
plot!(legend=:topleft, title = "Electricity Generation - Optimized (no HP)", legendcolumns=1, leftmargin=5mm, bottommargin=3mm)
plot!(p, size=(900, 450))

## Natural Gas, No Heat Pumps

#### Electricity and heat demand as is (with no heat pumps)

In [ ]:
# Optimizing cost of production for electricity and heat - no heat pumps
# define sets
G = 1:nrow(gens)
H = 1:1
T = 1:nrow(edemand)
NSECost = 10000
NSHCost = 5000

gencap = Model(HiGHS.Optimizer)
# define variables
@variables(gencap, begin
    y[g in G, t in T] >= 0
    z[h in H, t in T] >= 0
    NSE[t in T] >= 0
    NSH[t in T] >= 0
end)

@objective(gencap, Min, 
   sum(gens[G, :VarCost] .* sum(y[:, t] for t in T)) + NSECost * sum(NSE) + NSHCost * sum(NSH)
)

# @constraint(gencap, load[t in T], sum(y[:, t]) + NSE[t] >= edemand.edemand[t])
@constraint(gencap, load[t in T], sum(y[:, t]) + NSE[t] >= edemand.edemand[t])
@constraint(gencap, heating[t in T], sum(z[:, t]) + NSH[t] >= hdemand.hdemand[t])
@constraint(gencap, chp[t in T], y[1, t] >= heat[1, :Conversion_Factor]*z[1, t] )

@constraint(gencap, availability[g in G, t in T], y[g, t] <= gens[g, :Capacity]*cap_factor[t,g])
optimize!(gencap)

In [ ]:
@show objective_value(gencap)

In [ ]:
gens

In [ ]:
sum(value.(NSH).data)

In [ ]:
sum(value.(NSH).data)/sum(value.(y).data)

In [ ]:
# THis is the plot of electricity generation with heat pumps so demand is higher
# constraint to meet both electricity and heat demand

# remake for more hours
nsh = value.(NSH).data 
p = areaplot(nsh[1:200,:], 
    # label=permutedims(gens[:, :Column1]), 
    xlabel = "Hour", 
    # ylabel ="Generated Electricity (MW)", 
    color_palette=:mk_15,
    grid=:false,
    # ylim=(0, 100),
)
plot!(legend=:topleft, legendcolumns=1, title="Electricity Generation (no HP)", topmargin=5mm, leftmargin=5mm, bottommargin=5mm)
plot!(p, size=(850, 450))

In [ ]:
# THis is the plot of electricity generation with heat pumps so demand is higher
# constraint to meet both electricity and heat demand

# remake for more hours
gen = value.(y).data 
p = areaplot(gen'[1:200,:], 
    label=permutedims(gens[:, :Column1]), 
    xlabel = "Hour", 
    ylabel ="Generated Electricity (MW)", 
    color_palette=:mk_15,
    grid=:false,
    # ylim=(0, 100),
)
plot!(legend=:topleft, legendcolumns=1, title="Electricity Generation (no HP)", topmargin=5mm, leftmargin=5mm, bottommargin=5mm)
plot!(p, size=(850, 450))

## Natural Gas, Solar, & Heat Pumps

### Looking at Electricity & Heat

In [ ]:
# define sets
G = 1:nrow(gens)
H = 1:nrow(heat)
T = 1:nrow(edemand)
NSECost = 10000
NSHCost = 5000

gencap = Model(HiGHS.Optimizer)
# define variables
@variables(gencap, begin
    y[g in G, t in T] >= 0
    z[h in H, t in T] >= 0
    NSE[t in T] >= 0
    NSH[t in T] >= 0
end)

@objective(gencap, Min, 
   sum(gens[G, :VarCost] .* sum(y[:, t] for t in T)) + NSECost * sum(NSE) + NSHCost * sum(NSH)
)

# @constraint(gencap, load[t in T], sum(y[:, t]) + NSE[t] >= edemand.edemand[t])
@constraint(gencap, load[t in T], sum(y[:, t]) + NSE[t] >= edemand.edemand[t]+ heat[2, :Conversion_Factor]*z[2, t])
@constraint(gencap, heating[t in T], sum(z[:, t]) + NSH[t] >= hdemand.hdemand[t])
@constraint(gencap, chp[t in T], y[1, t] >= heat[1, :Conversion_Factor]*z[1, t] )

@constraint(gencap, availability[g in G, t in T], y[g, t] <= gens[g, :Capacity]*cap_factor[t,g])
optimize!(gencap)

In [ ]:
@show objective_value(gencap)

In [ ]:
heat_prod = @show value.(z)

In [ ]:
elec_prod = @show value.(y)

## Natural Gas + Heat Pumps

In [ ]:
# THis is the plot of electricity generation with heat pumps so demand is higher
# remake for more hours
gen = value.(y).data 
p = areaplot(gen'[1:200,:], 
    label=permutedims(gens[:, :Column1]), 
    xlabel = "Hour", 
    ylabel ="Generated Electricity (MW)", 
    color_palette=:mk_15,
    grid=:false,
    # ylim=(0, 100),
)
plot!(legend=:topleft, legendcolumns=1, title="Electricity Generation with HP", topmargin=5mm, leftmargin=5mm, bottommargin=5mm)
plot!(p, size=(850, 450))

In [ ]:
# Optimized cost with meeting electricity and heat demand
# remake for more hours
heat_data = value.(z).data 
p = areaplot(heat_data'[1:200,:], 
    label=permutedims(heat[:, :Column1]), 
    xlabel = "Hour", 
    ylabel ="Heat (MW)", 
    color_palette=:mk_15,
    grid=:false,
    # ylim=(0, 100),
)
plot!(legend=:topleft, legendcolumns=1, title="Heat Power", topmargin=5mm, leftmargin=5mm, bottommargin=5mm)
plot!(p, size=(850, 450))

In [ ]:
@show value(sum(NSE));

In [ ]:
chp_price = @show shadow_price.(chp)

In [ ]:
sum(chp_price)

In [ ]:
elec_price = @show shadow_price.(chp)